In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import openai
from sentence_transformers import SentenceTransformer, util
import json

states = ['Alabama', 'Arkansas', 'Kentucky', 'Louisiana', 'Mississippi', 'Tennessee', 'Texas', 'Oklahoma','California', 'Oregon', 'Washington', 'Alaska', 'Hawaii','Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico', 'Utah', 'Wyoming','Illinois', 'Indiana', 'Iowa', 'Kansas', 'Michigan', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'Ohio', 'South Dakota', 'Wisconsin','Pennsylvania', 'Vermont', 'West Virginia','District Of Columbia','Connecticut', 'Delaware', 'Florida', 'Georgia', 'Maine', 'Maryland', 'Massachusetts', 'New Hampshire', 'New Jersey', 'New York', 'North Carolina', 'Rhode Island', 'South Carolina', 'Virginia']


In [7]:

df = pd.read_csv(r'testing.csv')
print(df.iloc[5])
# print(df.iloc[0])
# print(df)


StartDate                2023-07-25 11:01:23
EndDate                  2023-07-25 11:02:05
Status                                     0
IPAddress                            *******
Progress                                 100
Duration (in seconds)                     41
Finished                                   1
RecordedDate             2023-07-25 11:02:06
ResponseId                 R_2AFOtRa3RA54r68
RecipientLastName                    *******
RecipientFirstName                   *******
RecipientEmail                       *******
ExternalReference                    *******
LocationLatitude                     *******
LocationLongitude                    *******
DistributionChannel                anonymous
UserLanguage                              EN
1_Q1                                       1
1_Q1_1_TEXT                               红叶
2_Q1                                       1
2_Q1_1_TEXT                              红叶园
3_Q1                                       1
3_Q1_1_TEX

### Sampling

In [ ]:
# Need to filter out non-English characters for the sample
def sample(state):
    filepath = 'filtered_datasets/states/'+state+'.csv'
    df = pd.read_csv(filepath, encoding = 'utf-8').dropna(subset=['rating', 'reviews'])
    df = df[df['name'].map(lambda x: x.isascii())]
    if df.shape[0] > 400:
        df_percent = df.sample(frac=0.1)
    elif 50 < df.shape[0] < 400:
        df_percent = df.sample(n = 50)
    else:
        df_percent = df
    df_percent = df_percent[['name', 'location_link','site','us_state','type','subtypes','Area','rating']]
#     df_percent.to_csv('samples/'+state+'.csv')
    return df_percent

def get_samples():
    samples = []
    for state in states:
        samples.append(sample(state))
    df_samples = pd.concat(samples).reset_index(drop = True)
    df_samples.to_csv('samples.csv')
    print(df_samples.shape[0])
    print(df_samples['us_state'].value_counts())
#     return df_samples
get_samples()

# def count(state):
#     filepath = 'samples/'+state+'.csv'
#     df = pd.read_csv(filepath, encoding = 'utf-8')
#     print(state+": "+str(df.shape[0]))

# def get_count():
#     for state in states:
#         count(state)
# get_count()

### We designed this to add some more samples to the sampled dataset in case we don't have enough number for a certain state.

In [3]:
def add_more(patch_path,sample_path, pool_path,state, number,other_path):
    sample = pd.read_csv(sample_path)
    pool = pd.read_csv(pool_path)
    other = pd.read_csv(other_path)
    try:
        patch_file = pd.read_csv(patch_path)
    except:
        patch_file = pd.DataFrame(columns = list(pool))
    # remove data in sample from pool
    pool = pool[~pool.google_id.isin(sample.google_id)]
    pool = pool[~pool.google_id.isin(other.google_id)]
    # print(pool.shape[0])
    patches = pool[pool['us_state']==state].sample(n = number)
    print(patches)
    print(type(patches))
    patch_file = pd.concat([patch_file,patches])
    patch_file.to_csv(patch_path,index=False,encoding='utf-8')



# add_more('utilities/patches.csv','output/sample_cleaned.csv','output/national.csv','Iowa',1)
# add_more('utilities/patches.csv','output/sample_cleaned.csv','output/national.csv','Utah',1)
# add_more('utilities/patches.csv','output/sample_cleaned.csv','output/national.csv','Alabama',1)
# add_more('utilities/patches.csv','output/sample_cleaned.csv','output/national.csv','Arkansas',1)
# add_more('utilities/patches.csv','output/sample_cleaned.csv','output/national.csv','Kansas',1)
# add_more('utilities/patches.csv','output/sample_cleaned.csv','output/national.csv','Kentucky',2)
add_more('utilities/patches.csv','output/sample_cleaned.csv','output/national.csv','Oklahoma',1,'utilities/patches.csv')

       Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  \
19762          9118          9118        9118   

                                           query               name site  \
19762  chinese restaurant, 74454, Porter, OK, US  East China Buffet  NaN   

                     type                              subtypes     category  \
19762  Chinese restaurant  Chinese restaurant, Asian restaurant  restaurants   

                 phone  ... booking_appointment_link menu_link order_links  \
19762  +1 918-485-3888  ...                      NaN       NaN         NaN   

                                           location_link  \
19762  https://www.google.com/maps/place/East+China+B...   

                          place_id                              google_id  \
19762  ChIJB3lqo7JytocRcWthes-Lfu4  0x87b672b2a36a7907:0xee7e8bcf7a616b71   

                        cid            reviews_id located_google_id  Area  
19762  17185326951320087409  -1261417122389464207               NaN     0  

[

/var/folders/ck/1xc3_hrd4556zhtcl0bbnq9r0000gn/T/ipykernel_2415/718746591.py:3: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  pool = pd.read_csv(pool_path)
